In [11]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [12]:
class autoencoder(object):
    
    def __init__(self, Training= True):
        self.isTrain = Training
        self.N_Pixels = 300*300
        
        self.Layer_1 = 50
        
        self.Latten_Dim = 2
        
        self.num_layer = 0
        
        self.ENDPOINT = {}
        
        self.two_layers()
            
    def weight_var(self, shape, name):
        initial = tf.truncated_normal(shape= shape, stddev= 0.1)
        return tf.Variable(initial, name= name)
    
    def bias_var(self, shape, name):
        initial = tf.truncated_normal(shape= shape, stddev= 0.1)
        return tf.Variable(initial, name= name)
    
    def full_connect_layer(self, x, w, b):
        return tf.matmul(x, w) + b
    
    def Layer(self, shape, inputs, name):
        self.num_layer += 1
        W = self.weight_var(shape, 'w_' + name + '_' + str(self.num_layer))
        B = self.bias_var([shape[1]], 'b_' + name + '_' + str(self.num_layer))
        return self.full_connect_layer(inputs, W, B)
    
    def two_layers(self):
        self.X = tf.placeholder(dtype= tf.float32, shape= [None, self.N_Pixels], name= 'X')
        # Encoder Layer 1
        self.ENDPOINT['enc_1'] = enc_1 = self.Layer([self.N_Pixels, self.Layer_1], self.X, 'enc')
        # Activation
        self.ENDPOINT['a_enc_1'] = a_enc_1 = tf.nn.tanh(enc_1)
        # Encoder Layer 2
        self.ENDPOINT['enc_2'] = enc_2 = self.Layer([self.Layer_1, self.Latten_Dim], a_enc_1, 'enc')
        
        self.ENDPOINT['logstd'] = logstd = self.Layer([self.Layer_1, self.Latten_Dim], a_enc_1, 'logstd')
        
        self.ENDPOINT['noise'] = noise = tf.random_normal([1, self.Latten_Dim])
        
        self.ENDPOINT['z'] = z = enc_2 + tf.multiply(noise, tf.exp(0.5 * logstd))
        
        # Decoder Layer 1
        self.ENDPOINT['dec_1'] = dec_1 = self.Layer([self.Latten_Dim, self.Layer_1], z, 'dec')
        # Activation
        self.ENDPOINT['a_dec_2'] = a_dec_2 = tf.nn.relu(dec_1)
        # Decoder Layer 2
        self.ENDPOINT['dec_2'] = dec_2 = self.Layer([self.Layer_1, self.N_Pixels], a_dec_2, 'dec')
        # Output Activation
        self.ENDPOINT['a_dec_1'] = a_dec_1 = tf.nn.sigmoid(dec_2)
        if self.isTrain:
            self.log_likelihood = self.Log_Likelihood(self.X, self.ENDPOINT['a_dec_1'])
            self.kl_divergence = self.KL_Divergence(self.ENDPOINT['enc_2'], self.ENDPOINT['logstd'])
            self.variation_lower_bound = self.Variation_Lower_Bound(self.log_likelihood, self.kl_divergence)
            tf.summary.scalar("Loss", self.log_likelihood)
            with tf.name_scope("Accuracy"):
                tf.summary.scalar("Accuracy", self.variation_lower_bound)
            self.optimizer = self.AOptimizer(self.variation_lower_bound)
        return a_dec_1
    
    def Log_Likelihood(self, inputs, a_dec_1):
        log_likelihood = tf.reduce_sum(inputs * tf.log(a_dec_1 + 1e-9) + (1 - inputs) * tf.log(1 - inputs + 1e-9), reduction_indices= 1)
        return log_likelihood
    
    def KL_Divergence(self, mean, std):
        kl_tern = -0.5 * tf.reduce_sum(1 + 2 * std - tf.pow(mean, 2) - tf.exp(2 * std), reduction_indices= 1)
        return kl_tern
    
    def Variation_Lower_Bound(self, Log_Likelihood, KL_Divergence):
        return tf.reduce_mean(Log_Likelihood + KL_Divergence)
    
    def Optimizer(self, loss):
        optimizer = tf.train.AdadeltaOptimizer().minimize(loss)
        return optimizer
    
    def AOptimizer(self, loss):
        with tf.name_scope("Adagrad"):
            optimizer = tf.train.AdagradOptimizer(0.1).minimize(loss)
        return optimizer
    

In [13]:
model_dir = os.path.join(os.getcwd(), "model")
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
MODEL_NAME = "CNN.ckpt"
LOG_DIR = "TensorBoard/"
SAVE_PATH = os.path.join(model_dir, MODEL_NAME)
Training = True

Name = 'Bus'
cur_dir = os.getcwd()
dataset = os.path.join(cur_dir, 'dataset')
dataset_dir = os.path.join(dataset, Name)
subimages = os.path.join(dataset_dir, 'SubImages')
images_path = []
if os.path.exists(subimages):
    images_name = os.listdir(subimages)
    for name in images_name:
        images_path.append(os.path.join(subimages, name))

auto = autoencoder(Training)

In [14]:
SAVER = tf.train.Saver()
init = tf.global_variables_initializer()
cfg = tf.ConfigProto(log_device_placement= False, allow_soft_placement= True)
cfg.gpu_options.per_process_gpu_memory_fraction = 0.90
cfg.gpu_options.allow_growth = True
cfg.gpu_options.allocator_type ='BFC'
sess = tf.InteractiveSession(config= cfg)
sess.run(init)

In [15]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(LOG_DIR, graph = sess.graph)

In [16]:
optimizer = auto.optimizer
variational_lower_bound = auto.variation_lower_bound
log_likelihood = auto.log_likelihood
KL_tern = auto.kl_divergence

In [17]:
if not Training:
    models = os.listdir(MODEL_SAVE_PATH)
    for file in models:
        if '.ckpt' in file:
            SAVER.restore(sess, SAVE_PATH)
            break
    img = cv2.imread(images_path[0])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_shape = img.shape
    img = cv2.resize(img, (300, 300), interpolation= cv2.INTER_LANCZOS4)
    img = np.reshape(img, [1, img.size])
    img = img / 255
    X = auto.X
    code = auto.ENDPOINT['logstd'].eval(feed_dict = {X: img})
    show_img = auto.ENDPOINT['a_dec_1'].eval(feed_dict = {X: img})
    show_img *= 255
    show_img = np.reshape(show_img, (300, 300))
    show_img = cv2.resize(show_img, (img_shape[1], img_shape[0]), interpolation= cv2.INTER_LANCZOS4)
    plt.imshow(show_img, cmap= 'gray')
    plt.show()
    print (code)

In [18]:
num_iterations = len(images_path)
recording_interval = 4
variational_lower_bound_array = []
log_likelihood_array = []
KL_tern_array = []
interation_array = [i * recording_interval for i in range(int(num_iterations / recording_interval))]
print (f"Images Length : {len(images_path)} \nTotal Interations : {num_iterations}\nInterval : {recording_interval}")
for i in range(num_iterations):
    img = cv2.imread(images_path[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_shape = img.shape
    img = cv2.resize(img, (300, 300), interpolation= cv2.INTER_LANCZOS4)
    img = np.reshape(img, [1, img.size])
    img = img / 255
    X = auto.X
    sess.run(auto.optimizer, feed_dict = {X: img})
    code = auto.ENDPOINT['logstd'].eval(feed_dict = {X: img})
    if (i % recording_interval == 0):
        vlb_eval = variational_lower_bound.eval(feed_dict = {X: img})
        print ("Iteration : {}, Loss : {}".format(i, vlb_eval))
        print (f"Code : {code}")
        variational_lower_bound_array.append(vlb_eval)
        log_likelihood_array.append(np.mean(log_likelihood.eval(feed_dict = {X: img})))
        KL_tern_array.append(np.mean(KL_tern.eval(feed_dict = {X: img})))
        #show_img = auto.ENDPOINT['a_dec_1'].eval(feed_dict = {X: img})
        #show_img *= 255
        #show_img = np.reshape(show_img, (300, 300))
        #show_img = cv2.resize(show_img, (img_shape[1], img_shape[0]), interpolation= cv2.INTER_LANCZOS4)
        #plt.imshow(show_img, cmap= 'gray')
        #plt.show()

Images Length : 432 
Total Interations : 432
Interval : 4
Iteration : 0, Loss : -40901.765625
Code : [[3.9916995 4.973943 ]]
Iteration : 4, Loss : -964647.3125
Code : [[-4.341372    0.54623634]]
Iteration : 8, Loss : -910697.25
Code : [[-4.3849287  0.9807164]]
Iteration : 12, Loss : -1252318.75
Code : [[-4.3840036  0.9805078]]
Iteration : 16, Loss : -1075904.75
Code : [[-4.364463  1.142786]]
Iteration : 20, Loss : -734371.75
Code : [[-4.372958   0.7141361]]
Iteration : 24, Loss : -979737.125
Code : [[-4.372049   0.7140163]]
Iteration : 28, Loss : -767053.0625
Code : [[-4.3711486   0.71389353]]
Iteration : 32, Loss : -553687.5625
Code : [[-4.155328    0.31772453]]
Iteration : 36, Loss : -1368376.375
Code : [[-4.154436    0.31761605]]
Iteration : 40, Loss : -1010307.1875
Code : [[-4.1534843   0.31757995]]
Iteration : 44, Loss : -660918.25
Code : [[-4.1525617   0.31754726]]
Iteration : 48, Loss : -540537.0
Code : [[-4.151634   0.3174643]]
Iteration : 52, Loss : -791793.5625
Code : [[-4.09

In [19]:
SAVER.save(sess, SAVE_PATH)
sess.close()

In [20]:
print (auto.isTrain)

True
